<a href="https://colab.research.google.com/github/lucprosa/dataeng-basic-course/blob/main/spark_streaming/examples/example_3_api_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up PySpark

In [1]:
%pip install pyspark

In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local').appName('Test streaming').getOrCreate()
sc = spark.sparkContext

In [5]:
!rm -rf /content/landing
!rm -rf /content/bronze
!mkdir -p /content/landing

## Simulate producer:
- extract data from API
- store data as json in the lake
- run task async

In [40]:
import requests
from pyspark.sql.types import *
import json
import datetime
import asyncio

async def ingest_from_api(url: str, table: str, schema: StructType = None):
  response = requests.get(url)
  timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
  if response.status_code == 200:
    data = response.json()
    with open(f"/content/landing/{table}_{int(timestamp)}.json", "w") as f:
        json.dump(data, f)

async def producer(loop: int, interval_time: int):
  for i in range(loop):
    await ingest_from_api("https://api.carrismetropolitana.pt/vehicles", "vehicles")
    await ingest_from_api("https://api.carrismetropolitana.pt/lines", "lines")
    await asyncio.sleep(interval_time)

async def main():
  asyncio.create_task(producer(10, 30))

await main()

- Read from /content/landing as streaming
- store data in memory (for testing)
- store data in the bronze layer

In [8]:
from pyspark.sql.types import *

vehicle_schema = StructType([StructField('bearing', IntegerType(), True),
                             StructField('block_id', StringType(), True),
                             StructField('current_status', StringType(), True),
                             StructField('id', StringType(), True),
                             StructField('lat', FloatType(), True),
                             StructField('line_id', StringType(), True),
                             StructField('lon', FloatType(), True),
                             StructField('pattern_id', StringType(), True),
                             StructField('route_id', StringType(), True),
                             StructField('schedule_relationship', StringType(), True),
                             StructField('shift_id', StringType(), True),
                             StructField('speed', FloatType(), True),
                             StructField('stop_id', StringType(), True),
                             StructField('timestamp', TimestampType(), True),
                             StructField('trip_id', StringType(), True)])

stream = spark.readStream.format("json").schema(vehicle_schema).load("/content/landing/vehicles*")

dedup = stream.dropDuplicates()

In [9]:
# using memory for testing
try:
  if query.isActive:
    query.stop()
except:
  pass

query = (dedup.writeStream.format("memory").option("queryName", "vehicles").start())

In [20]:
spark.sql("select * from vehicles").show()

+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+
|bearing|            block_id|current_status|      id|      lat|line_id|      lon|pattern_id|route_id|schedule_relationship|    shift_id|    speed|stop_id|          timestamp|             trip_id|
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+
|    209|       ESC_DU_EU2028|   INCOMING_AT| 43|2236|38.617523|   3012|-9.193602|  3012_0_2|  3012_0|            SCHEDULED|      EU2230|      7.5| 020417|2024-11-29 00:15:31|3012_0_2_2400_242...|
|      0|                1060|    STOPPED_AT| 42|1060| 38.83711|   2751|-9.187792|  2751_1_2|  2751_1|            SCHEDULED|       39743|      0.0| 070021|2024-11-29 00:16:12|2751_1_2|140|1|23...|
|    120|      

In [50]:
!rm -rf /content/bronze

In [48]:
from pyspark.sql.functions import *

# watermark is necessary because of the aggregation
transformed = stream.withWatermark("timestamp", "60 seconds")
agg = (transformed
       .groupBy(window(transformed.timestamp, "5 minutes"), col("current_status"))
       .agg(min(transformed.timestamp).alias("init_timestamp"), count("*").alias("count")))

def insert_vehicles(df, batch_id):
  #df2 = df.groupBy("window").pivot("current_status").sum("count")
  df.write.format("parquet").mode("append").save("/content/bronze/vehicles")

# using memory for testing
query2 = (agg
          .writeStream
          .outputMode("append")
          .foreachBatch(insert_vehicles)
          .option("checkpointLocation", "/content/bronze/checkpoint")
          .trigger(processingTime='20 seconds')
          .start())

In [44]:
spark.read.format("parquet").load("/content/bronze/vehicles/*").show(100, False)

+------------------------------------------+--------------+-------------------+-----+
|window                                    |current_status|init_timestamp     |count|
+------------------------------------------+--------------+-------------------+-----+
|{2024-11-29 00:25:00, 2024-11-29 00:30:00}|IN_TRANSIT_TO |2024-11-29 00:29:47|5    |
|{2024-11-29 00:10:00, 2024-11-29 00:15:00}|IN_TRANSIT_TO |2024-11-29 00:14:31|18   |
|{2024-11-29 00:15:00, 2024-11-29 00:20:00}|IN_TRANSIT_TO |2024-11-29 00:15:02|916  |
|{2024-11-29 00:20:00, 2024-11-29 00:25:00}|IN_TRANSIT_TO |2024-11-29 00:20:00|2    |
|{2024-11-29 00:15:00, 2024-11-29 00:20:00}|INCOMING_AT   |2024-11-29 00:15:01|308  |
|{2024-11-29 00:25:00, 2024-11-29 00:30:00}|INCOMING_AT   |2024-11-29 00:29:13|6    |
|{2024-11-29 00:10:00, 2024-11-29 00:15:00}|INCOMING_AT   |2024-11-29 00:14:29|4    |
|{2024-11-29 00:20:00, 2024-11-29 00:25:00}|INCOMING_AT   |2024-11-29 00:20:00|2    |
|{2024-11-29 00:15:00, 2024-11-29 00:20:00}|STOPPED_AT

## Report
- show vehicles by status in 5-min window time
- one line per window time

In [45]:
def pivot_data(df: DataFrame):
  result = df.orderBy("init_timestamp").groupBy("window").pivot("current_status").sum("count")
  result.show(100, False)

df = spark.read.format("parquet").load("/content/bronze/vehicles/*")
pivot_data(df)

+------------------------------------------+-----------+-------------+----------+
|window                                    |INCOMING_AT|IN_TRANSIT_TO|STOPPED_AT|
+------------------------------------------+-----------+-------------+----------+
|{2024-11-29 00:20:00, 2024-11-29 00:25:00}|2          |2            |NULL      |
|{2024-11-29 00:15:00, 2024-11-29 00:20:00}|308        |916          |295       |
|{2024-11-29 00:10:00, 2024-11-29 00:15:00}|4          |18           |16        |
|{2024-11-29 00:25:00, 2024-11-29 00:30:00}|6          |5            |10        |
+------------------------------------------+-----------+-------------+----------+



In [49]:
query2.stop()

ERROR:py4j.clientserver:There was an exception while executing the Python Proxy on the Python Side.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 617, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/utils.py", line 120, in call
    raise e
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/utils.py", line 117, in call
    self.func(DataFrame(jdf, wrapped_session_jdf), batch_id)
  File "<ipython-input-48-9446b2336b0e>", line 11, in insert_vehicles
    df.write.format("parquet").mode("append").save("/content/bronze/vehicles")
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/readwriter.py", line 1463, in save
    self._jwrite.save(path)
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
  File "/usr/local/lib/python3.10/dist-packages/pysp